In [1]:
#!pip install pandas-gbq

In [1]:
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
import requests
import datetime
import re
import json
from datetime import date, timedelta
#client = bigquery.Client(location="us-central1")
client = bigquery.Client(location="US")
#print("Client creating using default project: {}".format(client.project))
pd.set_option('display.width', 1000)
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, expr,first, last,when, split, col,lit, concat, date_format,to_utc_timestamp,from_utc_timestamp,to_timestamp, regexp_replace
from pyspark.sql.functions import sum as _sum


In [2]:
spark = SparkSession.builder.appName("StorageAllocation").getOrCreate()

In [3]:
current_date = datetime.datetime.today()

dias_atras = 3
#dias_atras = int(dias_atras)

str_day_process = (current_date - timedelta(days = dias_atras)).strftime("%Y-%m-%d")
str_day_process_dt = pd.to_datetime(str_day_process)
str_lastday = (str_day_process_dt - timedelta(days = 1)).strftime("%Y-%m-%d")
str_today = (str_day_process_dt + timedelta(days = 1)).strftime("%Y-%m-%d")

#STR_YEARMONTH = (current_date - timedelta(days = dias_atras)).strftime("%Y%m")
#str_day_process = "2022-07-25"
#STR_YEARMONTH = "201908"
print("Dia Analizado --> ",str_day_process," ; ","extraido de :",str_lastday," - ",str_today)

Dia Analizado -->  2023-05-21  ;  extraido de : 2023-05-20  -  2023-05-22


## YT1 

In [4]:
environment = 'YT1'

### Read YT1 Mongo DB Collection

In [27]:
df_mongo_yt1 = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','raw.yt1_mongodb_collection')\
.option("filter","date between '%s 00:00:00' and '%s 23:59:59' "%(str_day_process,str_day_process))\
.load()

df_mongo_yt1 = df_mongo_yt1.withColumn("environment",lit(environment))
df_mongo_yt1 = df_mongo_yt1.withColumn('date_normalized', date_format(to_date(col("date")),"yyyy-MM-dd"))
df_mongo_agg = df_mongo_yt1.groupBy(['date_normalized','tenant','real_name','environment'])\
                        .agg(_sum('size_db').alias('size_db'), \
                        _sum('size_storage').alias('size_storage'), \
                        _sum('size_index').alias('size_index'))

In [28]:
df_mongo_agg.show()

+---------------+--------+------------------+-----------+-------+-----------------+----------+
|date_normalized|  tenant|         real_name|environment|size_db|     size_storage|size_index|
+---------------+--------+------------------+-----------+-------+-----------------+----------+
|     2023-05-21|  FALACO|               SOH|        YT1|145.737|78.98100000000001|    18.811|
|     2023-05-21|  FALAPE|  soh_transactions|        YT1|  0.019|            0.004|      0.01|
|     2023-05-21|  CELINE|           PRINTER|        YT1|    0.0|              0.0|       0.0|
|     2023-05-21| MOJIXQA|         DD_SORTBY|        YT1|    0.0|              0.0|       0.0|
|     2023-05-21|    RSTQ|         TRANSFERS|        YT1|    0.0|              0.0|       0.0|
|     2023-05-21|    FALA|    thingTypeAlias|        YT1|    0.0|              0.0|       0.0|
|     2023-05-21|    SHOW|   DISCREPANCYEXEC|        YT1|    0.0|              0.0|       0.0|
|     2023-05-21|    SHOW|expiration_reflist|     

### YT1 Storage Cost Extraction - GCP

In [29]:
environment = 'YT1'

In [30]:
project_number = '260718309650'

In [31]:
GCP_cost_table = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','finance.gcp_billing_export_v1_01FE23_B1D2D8_10D34D')\
.option("filter","JSON_VALUE(project.number) = '%s' and usage_start_time between '%s 00:00:00' and '%s 23:59:59' "%(project_number,str_lastday, str_today))\
.load()

In [32]:
GCP_cost_table = GCP_cost_table.select(col("service.description").alias('service_description'),
                                          col("sku.description").alias('sku_description'),
                                          col("usage_start_time"),
                                          col("project.id").alias('id_project'),
                                          col("cost"),
                                          col("usage.amount").alias('usage_amount'),
                                          col("usage.unit").alias('usage_unit'),
                                          col("credits")[0].alias('credits_1'),
                                          col("credits")[1].alias('credits_2'),
                                          col("cost_type")
                                         )   

GCP_cost_table = GCP_cost_table.select(col('service_description'),
                                          col('sku_description'),
                                          col("usage_start_time"),
                                          col('id_project'),
                                          col("cost"),
                                          col('usage_amount'),
                                          col('usage_unit'),
                                          col("credits_1.name").alias('credits_1_name'),
                                          col("credits_1.amount").alias('credits_1_amount'),
                                          col("credits_1.type").alias('credits_1_type'),
                                          col("credits_2.name").alias('credits_2_name'),
                                          col("credits_2.amount").alias('credits_2_amount'),
                                          col("credits_2.type").alias('credits_2_type'),
                                          col("cost_type")
                                         ) 
#GCP_cost_table.show()

In [33]:
GCP_cost_table = GCP_cost_table.withColumn("credits_discounts",when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%DISCOUNT%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%DISCOUNT%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))

GCP_cost_table = GCP_cost_table.withColumn("credits_promotion",when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%PROMOTION%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%PROMOTION%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))
GCP_cost_table = GCP_cost_table.na.fill(value=0,subset=["credits_discounts",'credits_promotion'])

In [34]:
GCP_cost_table = GCP_cost_table.withColumn("storage_cost",when((GCP_cost_table['sku_description'].like('%SSD%')),GCP_cost_table['cost']))
GCP_cost_table = GCP_cost_table.withColumn("storage_discount",when((GCP_cost_table['sku_description'].like('%SSD%')),GCP_cost_table['credits_discounts']))
GCP_cost_table = GCP_cost_table.withColumn('total_storage_after_discount', GCP_cost_table['storage_cost'] + GCP_cost_table['credits_discounts'])


In [35]:
GCP_cost_table = GCP_cost_table.filter((GCP_cost_table['sku_description'].like('%SSD%')))

In [36]:
GCP_cost_table_agg = GCP_cost_table.groupBy(['usage_start_time'])\
                        .agg(_sum('storage_cost').alias('daily_storage_cost_sum'), \
                        _sum('total_storage_after_discount').alias('daily_storage_after_discount_sum'), \
                        _sum('credits_promotion').alias('daily_promotion_credits'))

df_cost_storage_gcp = GCP_cost_table.groupBy(['usage_start_time','sku_description'])\
                        .agg(_sum('storage_cost').alias('daily_storage_cost_sum'), \
                        _sum('storage_discount').alias('daily_storage_discount_sum'), \
                        _sum('total_storage_after_discount').alias('daily_storage_after_discount_sum'))

df_cost_storage_gcp = df_cost_storage_gcp.withColumn('date_normalized', date_format(to_date(from_utc_timestamp(col("usage_start_time"),"America/Los_Angeles")),"yyyy-MM-dd"))

df_cost_storage_gcp_agg = df_cost_storage_gcp.filter((df_cost_storage_gcp['date_normalized']==str_day_process)).groupBy(['date_normalized'])\
                        .agg(_sum('daily_storage_cost_sum').alias('daily_storage_cost_sum'), \
                        _sum('daily_storage_discount_sum').alias('daily_storage_discount_sum'), \
                        _sum('daily_storage_after_discount_sum').alias('daily_storage_after_discount_sum'))



In [37]:
df_cost_storage_gcp.show()

+-------------------+--------------------+----------------------+--------------------------+--------------------------------+---------------+
|   usage_start_time|     sku_description|daily_storage_cost_sum|daily_storage_discount_sum|daily_storage_after_discount_sum|date_normalized|
+-------------------+--------------------+----------------------+--------------------------+--------------------------------+---------------+
|2023-05-22 02:00:00|SSD backed PD Cap...|    18.224272999999997|                       0.0|              18.224272999999997|     2023-05-21|
|2023-05-22 10:00:00|SSD backed PD Cap...|    18.224274000000005|                       0.0|              18.224274000000005|     2023-05-22|
|2023-05-21 15:00:00|SSD backed Local ...|              0.120966|                       0.0|                        0.120966|     2023-05-21|
|2023-05-21 14:00:00|SSD backed PD Cap...|    18.224272000000006|                       0.0|              18.224272000000006|     2023-05-21|
|2023-

### YT1 JOIN Storage Usage with GCP Costs

In [38]:
df_results = df_mongo_agg.join(df_cost_storage_gcp_agg,['date_normalized'],"left")
df_results = df_results.join(df_mongo_agg.groupBy('date_normalized').agg(_sum('size_db').alias('size_db_tot_day'), \
                        _sum('size_storage').alias('size_storage_tot_day'), \
                        _sum('size_index').alias('size_index_tot_day'))\
                             ,['date_normalized'],"left")


df_results = df_results.withColumn("storage_cost_assigned",((df_results['daily_storage_cost_sum']/df_results['size_storage_tot_day'])*df_results['size_storage']))
df_results = df_results.withColumn("storage_cost_assigned_after_discounts",((df_results['daily_storage_after_discount_sum']/df_results['size_storage_tot_day'])*df_results['size_storage']))
df_results = df_results.withColumn("size_storage_percentage",((df_results['size_storage']/df_results['size_storage_tot_day'])*100))


df_results_upload = df_results[['environment','date_normalized','tenant','real_name',
                                'size_storage','size_storage_percentage',
                                'storage_cost_assigned','storage_cost_assigned_after_discounts']]


bucket = "finops-outputs"
spark.conf.set('temporaryGcsBucket', bucket)
print("Write data in Big Query Tables")

#df_results_upload.write.format('bigquery') \
 #   .option('table', 'saas-analytics-io.processed.storage_cost_assign_v2') \
  #  .mode('append') \
   # .save()

Write data in Big Query Tables


In [39]:
df_results_upload.show()

+-----------+---------------+--------+------------------+-----------------+-----------------------+---------------------+-------------------------------------+
|environment|date_normalized|  tenant|         real_name|     size_storage|size_storage_percentage|storage_cost_assigned|storage_cost_assigned_after_discounts|
+-----------+---------------+--------+------------------+-----------------+-----------------------+---------------------+-------------------------------------+
|        YT1|     2023-05-21|  FALACO|               SOH|78.98100000000001|    0.31611378207020374|   1.3918038983075083|                   1.3918038983075083|
|        YT1|     2023-05-21|  FALAPE|  soh_transactions|            0.004|   1.600961153037837...| 7.048803627745955E-5|                 7.048803627745955E-5|
|        YT1|     2023-05-21|  CELINE|           PRINTER|              0.0|                    0.0|                  0.0|                                  0.0|
|        YT1|     2023-05-21| MOJIXQA|  

## VT1 

In [40]:
environment = 'VT1'

### Read VT1 Mongo DB Collection

In [41]:
df_mongo_vt1 = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','raw.vt1_mongodb_collection')\
.option("filter","date between '%s 00:00:00' and '%s 23:59:59' "%(str_day_process,str_day_process))\
.load()

df_mongo_vt1 = df_mongo_vt1.withColumn("environment",lit(environment))
df_mongo_vt1 = df_mongo_vt1.withColumn('date_normalized', date_format(to_date(col("date")),"yyyy-MM-dd"))
df_mongo_agg = df_mongo_vt1.groupBy(['date_normalized','tenant','real_name','environment'])\
                        .agg(_sum('size_db').alias('size_db'), \
                        _sum('size_storage').alias('size_storage'), \
                        _sum('size_index').alias('size_index'))

### VT1 Storage Cost Extraction - GCP

In [42]:
environment = 'YT1'

In [43]:
project_number = '884190693971'

In [44]:
GCP_cost_table = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','finance.gcp_billing_export_v1_01FE23_B1D2D8_10D34D')\
.option("filter","JSON_VALUE(project.number) = '%s' and usage_start_time between '%s 00:00:00' and '%s 23:59:59' "%(project_number,str_lastday, str_today))\
.load()

In [45]:
GCP_cost_table = GCP_cost_table.select(col("service.description").alias('service_description'),
                                          col("sku.description").alias('sku_description'),
                                          col("usage_start_time"),
                                          col("project.id").alias('id_project'),
                                          col("cost"),
                                          col("usage.amount").alias('usage_amount'),
                                          col("usage.unit").alias('usage_unit'),
                                          col("credits")[0].alias('credits_1'),
                                          col("credits")[1].alias('credits_2'),
                                          col("cost_type")
                                         )   

GCP_cost_table = GCP_cost_table.select(col('service_description'),
                                          col('sku_description'),
                                          col("usage_start_time"),
                                          col('id_project'),
                                          col("cost"),
                                          col('usage_amount'),
                                          col('usage_unit'),
                                          col("credits_1.name").alias('credits_1_name'),
                                          col("credits_1.amount").alias('credits_1_amount'),
                                          col("credits_1.type").alias('credits_1_type'),
                                          col("credits_2.name").alias('credits_2_name'),
                                          col("credits_2.amount").alias('credits_2_amount'),
                                          col("credits_2.type").alias('credits_2_type'),
                                          col("cost_type")
                                         ) 
#GCP_cost_table.show()

In [46]:
GCP_cost_table = GCP_cost_table.withColumn("credits_discounts",when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%DISCOUNT%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%DISCOUNT%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))

GCP_cost_table = GCP_cost_table.withColumn("credits_promotion",when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%PROMOTION%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%PROMOTION%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))
GCP_cost_table = GCP_cost_table.na.fill(value=0,subset=["credits_discounts",'credits_promotion'])

In [47]:
GCP_cost_table = GCP_cost_table.withColumn("storage_cost",when((GCP_cost_table['sku_description'].like('%SSD%')),GCP_cost_table['cost']))
GCP_cost_table = GCP_cost_table.withColumn("storage_discount",when((GCP_cost_table['sku_description'].like('%SSD%')),GCP_cost_table['credits_discounts']))
GCP_cost_table = GCP_cost_table.withColumn('total_storage_after_discount', GCP_cost_table['storage_cost'] + GCP_cost_table['credits_discounts'])


In [48]:
GCP_cost_table = GCP_cost_table.filter((GCP_cost_table['sku_description'].like('%SSD%')))

In [49]:
GCP_cost_table_agg = GCP_cost_table.groupBy(['usage_start_time'])\
                        .agg(_sum('storage_cost').alias('daily_storage_cost_sum'), \
                        _sum('total_storage_after_discount').alias('daily_storage_after_discount_sum'), \
                        _sum('credits_promotion').alias('daily_promotion_credits'))

df_cost_storage_gcp = GCP_cost_table.groupBy(['usage_start_time','sku_description'])\
                        .agg(_sum('storage_cost').alias('daily_storage_cost_sum'), \
                        _sum('storage_discount').alias('daily_storage_discount_sum'), \
                        _sum('total_storage_after_discount').alias('daily_storage_after_discount_sum'))

df_cost_storage_gcp = df_cost_storage_gcp.withColumn('date_normalized', date_format(to_date(from_utc_timestamp(col("usage_start_time"),"America/Los_Angeles")),"yyyy-MM-dd"))

df_cost_storage_gcp_agg = df_cost_storage_gcp.filter((df_cost_storage_gcp['date_normalized']==str_day_process)).groupBy(['date_normalized'])\
                        .agg(_sum('daily_storage_cost_sum').alias('daily_storage_cost_sum'), \
                        _sum('daily_storage_discount_sum').alias('daily_storage_discount_sum'), \
                        _sum('daily_storage_after_discount_sum').alias('daily_storage_after_discount_sum'))


### VT1 JOIN Storage Usage with GCP Costs

In [50]:
df_results = df_mongo_agg.join(df_cost_storage_gcp_agg,['date_normalized'],"left")
df_results = df_results.join(df_mongo_agg.groupBy('date_normalized').agg(_sum('size_db').alias('size_db_tot_day'), \
                        _sum('size_storage').alias('size_storage_tot_day'), \
                        _sum('size_index').alias('size_index_tot_day'))\
                             ,['date_normalized'],"left")


df_results = df_results.withColumn("storage_cost_assigned",((df_results['daily_storage_cost_sum']/df_results['size_storage_tot_day'])*df_results['size_storage']))
df_results = df_results.withColumn("storage_cost_assigned_after_discounts",((df_results['daily_storage_after_discount_sum']/df_results['size_storage_tot_day'])*df_results['size_storage']))
df_results = df_results.withColumn("size_storage_percentage",((df_results['size_storage']/df_results['size_storage_tot_day'])*100))


df_results_upload = df_results[['environment','date_normalized','tenant','real_name',
                                'size_storage','size_storage_percentage',
                                'storage_cost_assigned','storage_cost_assigned_after_discounts']]


#bucket = "finops-outputs"
#spark.conf.set('temporaryGcsBucket', bucket)
#print("Write data in Big Query Tables")
#
#df_results_upload.write.format('bigquery') \
#    .option('table', 'saas-analytics-io.processed.storage_cost_assign_v2') \
#    .mode('append') \
#    .save()

In [51]:
df_results_upload.show()

+-----------+---------------+----------+--------------------+--------------------+-----------------------+---------------------+-------------------------------------+
|environment|date_normalized|    tenant|           real_name|        size_storage|size_storage_percentage|storage_cost_assigned|storage_cost_assigned_after_discounts|
+-----------+---------------+----------+--------------------+--------------------+-----------------------+---------------------+-------------------------------------+
|        VT1|     2023-05-21|    LUERSA|            EMPLOYEE|                 0.0|                    0.0|                  0.0|                                  0.0|
|        VT1|     2023-05-21|    LUERSA|          vizixEvent|               0.004|   1.974762534805189...| 3.757776219909555...|                 3.757776219909555...|
|        VT1|     2023-05-21|     MOJIX|          statuscube|                 0.0|                    0.0|                  0.0|                                  0.0